In [ ]:
import sys
sys.path.insert(0, '../scripts/')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from itertools import count
import random
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from models import AE, BaselineNet
from utils import *

import seaborn as sns

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from sklearn.ensemble import ExtraTreesClassifier

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder

color = sns.color_palette()

In [ ]:
df = pd.read_csv('../data/denver_prop_with_ct_clean.csv')

In [ ]:
df = df.astype({
  'list_date': 'datetime64[ns]',
  'sale_date': 'datetime64[ns]'
})
df = df.drop(columns=['rex_property_id'])

In [ ]:
res = gen_dataset(df, '2017-04-02', 90)

In [ ]:
X = res['X']
y = res['y']
print('num float: ' + str(len(res['float'])))
print('num bool: ' + str(len(res['bool'])))
print('num int: ' + str(len(res['int'])))

num float: 45
num bool: 9
num int: 10


In [ ]:
X_float = X[res['float']]

In [ ]:
X_float

,bathtotalcalc,median_rental_price,average_annual_rent_as_percentage_of_property_value,farm_score,pop_density,urban_score,rural_score,built 1995 or later,built 1970-1994,built 1940-1969,...,public_transportation,avg_household_size,percent_mortgage,built 2014 or later,built 2010-2013,built 2000-2009,<18,commute_under_30,18-59,sqft
5,3.5,2672.0,8.331314,100.0,5944.113744,2.0,68.0,80.577849,18.780096,0.642055,...,0.022361,3.42,0.936654,0.000000,0.000000,0.805778,0.326826,52.646544,0.616287,1678.0
6,4.5,2672.0,8.331314,100.0,5944.113744,2.0,68.0,80.577849,18.780096,0.642055,...,0.022361,3.42,0.936654,0.000000,0.000000,0.805778,0.326826,52.646544,0.616287,2996.0
10,2.0,2672.0,8.331314,100.0,5944.113744,2.0,68.0,80.577849,18.780096,0.642055,...,0.022361,3.42,0.936654,0.000000,0.000000,0.805778,0.326826,52.646544,0.616287,1400.0
24,3.5,2672.0,8.331314,100.0,5944.113744,2.0,68.0,80.577849,18.780096,0.642055,...,0.022361,3.42,0.936654,0.000000,0.000000,0.805778,0.326826,52.646544,0.616287,1469.0
45,3.5,2672.0,8.331314,100.0,5944.113744,2.0,68.0,80.577849,18.780096,0.642055,...,0.022361,3.42,0.936654,0.000000,0.000000,0.805778,0.326826,52.646544,0.616287,2635.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277543,2.5,1438.0,24.420792,50.0,2063.788745,34.0,36.0,33.745875,64.191419,0.495050,...,0.038314,3.82,0.497200,0.089109,0.134488,0.113861,0.338800,60.975610,0.576851,2550.0
277544,2.0,2628.0,7.660988,100.0,6679.282809,1.0,71.0,3.015682,94.270205,1.990350,...,0.020492,2.77,0.763071,0.000000,0.000000,0.030157,0.232877,60.092127,0.623831,1424.0
277546,2.0,2628.0,7.660988,100.0,6679.282809,1.0,71.0,3.015682,94.270205,1.990350,...,0.020492,2.77,0.763071,0.000000,0.000000,0.030157,0.232877,60.092127,0.623831,1424.0
277549,3.5,1821.0,4.200747,0.0,111.711091,64.0,6.0,40.666667,46.833333,3.444444,...,0.004424,2.92,0.666159,0.034722,0.068889,0.303056,0.266798,47.654827,0.519823,3169.0


In [ ]:
X_bool = X[res['bool']]

In [ ]:
X_bool

,CONDO,MULTI_FAMILY,OTHER,SINGLE,has_central_air,has_jacuzzi,has_pool,has_solar,has_garage
5,False,False,False,True,False,False,False,False,False
6,False,False,False,True,False,False,False,False,False
10,True,False,False,False,False,False,False,False,False
24,False,False,False,True,False,False,False,False,False
45,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
277543,False,False,False,True,False,False,False,False,False
277544,True,False,False,False,False,False,False,False,False
277546,True,False,False,False,False,False,False,False,False
277549,False,False,False,True,False,False,False,False,False


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1c850c61-d934-4c85-b16d-3cb283df0c84' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>